# Train and save a servable TensorFlow model

In [1]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))

TensorFlow version: 2.4.1


In [2]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras


## Create the model

### Import the banking dataset to detect churn activities

In [3]:
#importing the data
data = pd.read_csv("https://raw.githubusercontent.com/AdeloreSimiloluwa/Artificial-Neural-Network/master/data/Churn_Modelling.csv")
# data = pd.read_csv("Churn_Modelling.csv")

In [4]:
#dropping some columns that are not needed
data = data.drop(columns=['RowNumber','CustomerId','Surname'], axis=1)

#split data into features #target data
X, y  = data.iloc[:,:-1], data.iloc[:,-1:]


#encoding the categorical columns
le = LabelEncoder()
ohe = OneHotEncoder()
X['Gender'] = le.fit_transform(X['Gender'])
geo_df = pd.DataFrame(ohe.fit_transform(X[['Geography']]).toarray())

#getting feature name after onehotencoding
geo_df.columns = ohe.get_feature_names(['Geography'])

#merging geo_df with the main data
X = X.join(geo_df)

#dropping the old columns after encoding
X.drop(columns=['Geography'], axis=1, inplace=True)

In [5]:
#Split data into train and test set
X_train,X_test,y_train,y_test = train_test_split( X,y, test_size=0.2, random_state = 42)

In [6]:
data_dict = {"X_train":X_train, "X_test":X_test, "y_train":y_train, "y_test":y_test}
for i in data_dict:
    print("The shape of {} is {}".format(i,data_dict[i].shape))

The shape of X_train is (8000, 12)
The shape of X_test is (2000, 12)
The shape of y_train is (8000, 1)
The shape of y_test is (2000, 1)


In [7]:
# Feature Scaling
sc =StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Train and evaluate your model

#### Deep learning model 1

In [8]:
#initializing the classifier model with its input, hidden and output layers
classifier = keras.Sequential([
                          keras.layers.Dense(units =16, input_dim=12, kernel_initializer='uniform', activation='relu'),
                          keras.layers.Dense(units =8, kernel_initializer='uniform', activation='relu'),
                          keras.layers.Dense(units =1, kernel_initializer='uniform', activation='sigmoid')
])
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                208       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 353
Trainable params: 353
Non-trainable params: 0
_________________________________________________________________


In [9]:
### Compiling the classifier model with Stochastic Gradient Desecnt
classifier.compile(optimizer = 'adam', loss='binary_crossentropy' , metrics =['accuracy'])
### Fitting the classifier model
classifier.fit(X_train, y_train, batch_size=10 , epochs=5)

test_loss, test_acc = classifier.evaluate(X_test,y_test)
print('\nTest accuracy: {}'.format(test_acc))


Epoch 1/5
800/800 [==============================] - 2s 2ms/step - loss: 0.5452 - accuracy: 0.7922
Epoch 2/5
800/800 [==============================] - 2s 2ms/step - loss: 0.4365 - accuracy: 0.7918
Epoch 3/5
800/800 [==============================] - 1s 2ms/step - loss: 0.4215 - accuracy: 0.8094
Epoch 4/5
800/800 [==============================] - 1s 2ms/step - loss: 0.4282 - accuracy: 0.8168
Epoch 5/5
63/63 [==============================] - 0s 2ms/step - loss: 0.4003 - accuracy: 0.8310

Test accuracy: 0.8309999704360962


## Save your model

To load our trained model into TensorFlow Serving we first need to save it in [SavedModel](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/saved_model) format.  This will create a protobuf file in a well-defined directory hierarchy, and will include a version number.  [TensorFlow Serving](https://www.tensorflow.org/tfx/guide/serving) allows us to select which version of a model, or "servable" we want to use when we make inference requests.  Each version will be exported to a different sub-directory under the given path.

In [10]:
import tempfile

MODEL_DIR = tempfile.gettempdir()

version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    classifier,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}

export_path = /tmp/1

INFO:tensorflow:Assets written to: /tmp/1/assets

Saved model:
total 100
drwxr-xr-x 2 twarik twarik  4096 Feb  2 14:32 assets
-rw-r--r-- 1 twarik twarik 91204 Feb  6 01:18 saved_model.pb
drwxr-xr-x 2 twarik twarik  4096 Feb  6 01:18 variables


## Examine our saved model

In [11]:
!saved_model_cli show --dir {export_path} --all

2021-02-06 01:19:00.003846: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-02-06 01:19:00.003888: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['dense_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 12)
        name: serving_def